# import 

In [4]:
from F import F
from config import config
import joblib
import numpy as np
import pandas as pd
import os
print(os.getcwd())

/home/ubuntu/Desktop/c620


# demo data

In [5]:
tag = 'test001'
demo = joblib.load('./data/demo.pkl')
c620_case = demo['c620_case']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
t651_mf = demo['t651_mf'].to_frame()
c620_mf_side = demo['c620_mf_side'].to_frame()
c660_case = demo['c660_case']
c620_mf_bot = demo['c620_mf_bot'].to_frame()
c660_mf_bot = demo['c660_mf_bot'].to_frame()
c670_bf = demo['c670_bf']

# ICG

In [6]:
model_icg = joblib.load('model/c620_icg.pkl')
ICG_Input = pd.DataFrame(index=[tag],columns=model_icg.x_col)
for cname in ICG_Input.columns:
    min_ = model_icg.ss_x.data_min_[model_icg.x_col.index(cname)]
    max_ = model_icg.ss_x.data_max_[model_icg.x_col.index(cname)]
    mean_ = min_ + (max_ - min_)/2
    ICG_Input[cname] = mean_
ICG_Input

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
test001,181.5,1.236011,10.736613,44.816494,15.0,2.338977,74.302254,17.467825,62.0,0.27175,32.548999,59.738174,80.0,890.0


In [7]:
def icg_predict(icg_input):
    while True:
        output = model_icg.predict(icg_input)
        if np.allclose(output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0],0.01,atol=1e-2):
            info = 'current NA in Benzene_ppmw is:{} current Distillate Rate is:{} so NA in Benzene_ppmw -= 30'.format(
                icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0],
                output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0])
            icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] -= 30
            print(info)
        else:
            return output

In [8]:
icg_dist = icg_predict(ICG_Input)
icg_dist

current NA in Benzene_ppmw is:890.0 current Distillate Rate is:0.010853055864572525 so NA in Benzene_ppmw -= 30
current NA in Benzene_ppmw is:860.0 current Distillate Rate is:0.012139653787016869 so NA in Benzene_ppmw -= 30
current NA in Benzene_ppmw is:830.0 current Distillate Rate is:0.014894358813762665 so NA in Benzene_ppmw -= 30
current NA in Benzene_ppmw is:800.0 current Distillate Rate is:0.019841212779283524 so NA in Benzene_ppmw -= 30


,Simulation Case Conditions_C620 Distillate Rate_m3/hr
test001,0.028154


# ICG Distillate Rate 代入 c620_case

In [9]:
c620_case

,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%
028-050,0.25,70.0


In [10]:
c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_dist.values[0][0]

In [11]:
c620_case

,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%
028-050,0.028154,70.0


# F

In [12]:
f = F(config)
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f(c620_case,c620_feed,t651_feed,t651_mf,c620_mf_side,c660_case,c620_mf_bot,c660_mf_bot,c670_bf)

In [13]:
c620_wt

,Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Methane_wt%,Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Propane_wt%,Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Pentane_wt%,Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Hexane_wt%,Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Benzene_wt%,Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Cyclohexane_wt%,Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Heptane_wt%,...,Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Indane_wt%,Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_1-Methyl-4-n-propylbenzene_wt%,"Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_1,2-Diethylbenzene_wt%",Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_5-Ethyl-m-xylene_wt%,"Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_1,4-Diethylbenzene_wt%","Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_1,2,3,5-Tetramethylbenzene_wt%",Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Oxygen_wt%
028-050,0.0,1.637723,39.274569,35.696535,10.338633,4.709528,0.241827,8.052404,0.011544,0.002941,...,0.041828,0.001195,0.027487,1.074385,0.0,1.235722,0.316699,0.13624,0.0,0.0


In [14]:
c620_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate Production Rate and Composition,Density_Sidedraw Production Rate and Composition,Density_Bottoms Production Rate and Composition,Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
028-050,0.862281,0.452975,0.781814,0.878705,0.871925,104.628052,10.513339,12.119115,177.616104,193.987442


In [15]:
c660_wt

,Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Methane_wt%,Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Propane_wt%,Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Pentane_wt%,Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Hexane_wt%,Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Benzene_wt%,Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Cyclohexane_wt%,Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Heptane_wt%,...,Benzene Column C660 Operation_Bottoms Production Rate and Composition_Indane_wt%,Benzene Column C660 Operation_Bottoms Production Rate and Composition_1-Methyl-4-n-propylbenzene_wt%,"Benzene Column C660 Operation_Bottoms Production Rate and Composition_1,2-Diethylbenzene_wt%",Benzene Column C660 Operation_Bottoms Production Rate and Composition_5-Ethyl-m-xylene_wt%,"Benzene Column C660 Operation_Bottoms Production Rate and Composition_1,4-Diethylbenzene_wt%","Benzene Column C660 Operation_Bottoms Production Rate and Composition_1,2,3,5-Tetramethylbenzene_wt%",Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,Benzene Column C660 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,Benzene Column C660 Operation_Bottoms Production Rate and Composition_Oxygen_wt%
028-050,0.0,0.300251,17.942058,27.102143,10.893385,7.304881,1.819152,34.541724,0.071672,0.009928,...,3.588816e-09,0.037407,0.009321,1.316283e-08,0.0,1.673227e-10,7.753419e-12,8.457082e-15,0.0,0.0


In [16]:
c660_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate (Benzene Drag) Production Rate and Composition,Density_Sidedraw (Benzene )Production Rate and Composition,Density_Bottoms Production Rate and Composition,Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
028-050,0.876411,0.566776,0.85626,0.883697,0.870562,147.670776,14.034904,13.563004,86.716644,90.287651


In [17]:
c670_wt

,Toluene Column C670 Operation_Distillate Production Rate and Composition_Hydrogen_wt%,Toluene Column C670 Operation_Distillate Production Rate and Composition_Methane_wt%,Toluene Column C670 Operation_Distillate Production Rate and Composition_Ethane_wt%,Toluene Column C670 Operation_Distillate Production Rate and Composition_Propane_wt%,Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Butane_wt%,Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Pentane_wt%,Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Hexane_wt%,Toluene Column C670 Operation_Distillate Production Rate and Composition_Benzene_wt%,Toluene Column C670 Operation_Distillate Production Rate and Composition_Cyclohexane_wt%,Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Heptane_wt%,...,Toluene Column C670 Operation_Bottoms Production Rate and Composition_Indane_wt%,Toluene Column C670 Operation_Bottoms Production Rate and Composition_1-Methyl-4-n-propylbenzene_wt%,"Toluene Column C670 Operation_Bottoms Production Rate and Composition_1,2-Diethylbenzene_wt%",Toluene Column C670 Operation_Bottoms Production Rate and Composition_5-Ethyl-m-xylene_wt%,"Toluene Column C670 Operation_Bottoms Production Rate and Composition_1,4-Diethylbenzene_wt%","Toluene Column C670 Operation_Bottoms Production Rate and Composition_1,2,3,5-Tetramethylbenzene_wt%",Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,Toluene Column C670 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,Toluene Column C670 Operation_Bottoms Production Rate and Composition_Oxygen_wt%
028-050,0.0,1.853913e-07,0.000009,0.000007,8.759481e-24,2.821000e-16,2.306242e-09,0.306911,0.000255,0.00489,...,0.068608,0.028453,0.051687,1.762251,0.0,2.026864,0.519457,0.223465,0.0,0.0


In [18]:
c670_op

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
028-050,0.870914,0.8723,235.166962,27.25713,27.424627,181.185944,224.813736
